# Bradley Terry Model
 
This notebook consists of Four parts
1. Load relevant packages
2. Load data
3. Data Preprocessing
4. Model(Training/Evaluation)

To simplfy the first model and more interpratable result, the features we concerned here will only be recent xG, recent win pertages, rank and history records of two teams

## 1.Import Relevant Packages

In [134]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import *
import seaborn as sns
import random
import scipy
from utils import *
from patsy import dmatrices
import re

In [12]:
pip install patsy


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2.Import Data

### 2.1 Define some constant

In [135]:
team = 'Chelsea'
base_year = 2017
train_year = 2018

### 2.2 Load all the data

In [140]:

df_2017 = get_match_result(team, base_year)
df_2018 = get_match_result(team, train_year)
df_2016 = get_match_result(team, 2016)


In [141]:
df_2017

[{'id': '7123',
  'isResult': True,
  'side': 'h',
  'h': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'},
  'a': {'id': '92', 'title': 'Burnley', 'short_title': 'BUR'},
  'goals': {'h': '2', 'a': '3'},
  'xG': {'h': '1.35651', 'a': '0.564237'},
  'datetime': '2017-08-12 15:00:00',
  'forecast': {'w': 0.5634410014960609,
   'd': 0.28199586548919725,
   'l': 0.1545631329938977},
  'result': 'l'},
 {'id': '7137',
  'isResult': True,
  'side': 'a',
  'h': {'id': '82', 'title': 'Tottenham', 'short_title': 'TOT'},
  'a': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'},
  'goals': {'h': '1', 'a': '2'},
  'xG': {'h': '0.72428', 'a': '0.834423'},
  'datetime': '2017-08-20 16:00:00',
  'forecast': {'w': 0.2891247126912611,
   'd': 0.35812341200258585,
   'l': 0.3527518753061269},
  'result': 'w'},
 {'id': '7140',
  'isResult': True,
  'side': 'h',
  'h': {'id': '80', 'title': 'Chelsea', 'short_title': 'CHE'},
  'a': {'id': '72', 'title': 'Everton', 'short_title': 'EVE'},
  'goals'

In [137]:
df_2017, df_2018 = Constrcut_df_from_results(df_2017), Constrcut_df_from_results(df_2018)

In [142]:
df_2016 = Constrcut_df_from_results(df_2016)
df_2016.head()

,id,oppenet,opponent_id,home,result,datetime,xG,num_of_score_goals,num_of_conced_goals
0,470,West Ham,81,0,w,2016-08-15 23:00:00,1.4603,2,1
1,476,Watford,90,1,w,2016-08-20 18:00:00,1.75595,2,1
2,485,Burnley,92,0,w,2016-08-27 18:00:00,2.65838,3,0
3,499,Swansea,84,1,d,2016-09-11 19:00:00,2.19035,2,2
4,501,Liverpool,87,0,l,2016-09-16 23:00:00,1.16415,1,2


In [138]:
df_2017.head()

,id,oppenet,opponent_id,home,result,datetime,xG,num_of_score_goals,num_of_conced_goals
0,7123,Burnley,92,0,l,2017-08-12 15:00:00,1.35651,2,3
1,7137,Tottenham,82,1,w,2017-08-20 16:00:00,0.834423,2,1
2,7140,Everton,72,0,w,2017-08-27 13:30:00,1.16991,2,0
3,7151,Leicester,75,1,w,2017-09-09 15:00:00,1.03943,2,1
4,7167,Arsenal,83,0,d,2017-09-17 13:30:00,0.762129,0,0


In [63]:
df_2018.head()

,id,oppenet,opponent_id,home,result,datetime,xG,num_of_score_goals,num_of_conced_goals
0,9200,Huddersfield,219,1,w,2018-08-11 17:00:00,2.31427,3,0
1,9212,Arsenal,83,0,w,2018-08-18 19:30:00,1.71799,3,2
2,9225,Newcastle United,86,1,w,2018-08-26 18:00:00,1.49669,2,1
3,9231,Bournemouth,73,0,w,2018-09-01 14:00:00,1.16408,2,0
4,9241,Cardiff,227,0,w,2018-09-15 14:00:00,2.67145,4,1


## 3. Data Preprocessing

* Add rank of the table
* Add history record

### 3.1 Add Current Rank

In [74]:
# rank
def get_current_rank_data(league, season, start_date, end_date):
    async def main():
        async with aiohttp.ClientSession() as session:
            understat = Understat(session)
            table = await understat.get_league_table(league, season, start_date= start_date, end_date=end_date)
            return table

    loop = asyncio.get_event_loop()
    table = loop.run_until_complete(main())
    return table



In [133]:
import datetime


def get_current_rank(end_date, team, opponent, start_date, year):
    table = get_current_rank_data('EPL', year, start_date= start_date, end_date=end_date)
    data = pd.DataFrame(table[1:], columns=table[0])
    rank = data.loc[data['Team'] == team].index.to_list()[0] + 1
    opponent_rank = data.loc[data['Team'] == opponent].index.to_list()[0] + 1
    return rank, opponent_rank

def minus_one_day(dt):
    date = datetime.datetime.strptime(dt, '%Y-%m-%d') - datetime.timedelta(days = 1)
    return datetime.datetime.strftime(date,"%Y-%m-%d")


In [135]:
df_2017['team'], df_2018['team'] = team, team

df_2017[f'{team}_rank'], df_2017['opponent_rank'] = zip(*df_2017.apply(lambda x: get_current_rank(minus_one_day(x['datetime'].split(' ')[0]), x['team'], x['oppenet'], '2017-08-11', '2017'), axis = 1))

df_2018[f'{team}_rank'], df_2018['opponent_rank'] = zip(*df_2018.apply(lambda x: get_current_rank(minus_one_day(x['datetime'].split(' ')[0]), x['team'], x['oppenet'], '2018-08-10', '2018'), axis = 1))




In [137]:
df_2017.head(), df_2018.head()

(     id    oppenet opponent_id  home result             datetime        xG  \
 0  7123    Burnley          92     0      l  2017-08-12 15:00:00   1.35651   
 1  7137  Tottenham          82     1      w  2017-08-20 16:00:00  0.834423   
 2  7140    Everton          72     0      w  2017-08-27 13:30:00   1.16991   
 3  7151  Leicester          75     1      w  2017-09-09 15:00:00   1.03943   
 4  7167    Arsenal          83     0      d  2017-09-17 13:30:00  0.762129   
 
   num_of_score_goals num_of_conced_goals     team  Chelsea_rank  opponent_rank  
 0                  2                   3  Chelsea             7             17  
 1                  2                   1  Chelsea            15              7  
 2                  2                   0  Chelsea            11              7  
 3                  2                   1  Chelsea             6             15  
 4                  0                   0  Chelsea             3             12  ,
      id           oppenet oppo

### 3.2 Add history record

For generak ficture and data, **group by and agg can be used** to generate results

In [192]:
columns = df_2016.columns.to_list()

def get_win_percentage(idx, opponent, current_df, data):
    d = current_df.loc[:idx-1, columns]
    if idx != 0:
        data = pd.concat([data,d], axis = 0)
    else:
        pass
    data_dict = {'w':0, 'd':0, 'l': 0}
   
    if opponent in data['oppenet'].to_list():
        result = data.loc[data['oppenet'] == opponent, 'result'].to_list()
        print(opponent ,result)
        for res in result:
            if res == 'w':
                data_dict['w'] += 1
            elif res == 'd':
                data_dict['d'] += 1
            else:
                data_dict['l'] += 1
    else:
        pass
    print(data_dict)
    total = sum(data_dict.values())      
    return data_dict['w'] / total if total !=0 else 0.5


In [193]:
# add reocrd for 2017
data_2017 = df_2016
df_2017['Fixture_id'] = df_2017.index
df_2017['team_history_record'] = df_2017.apply(lambda x: get_win_percentage(x['Fixture_id'], x['oppenet'], df_2017, data_2017), axis = 1)

Burnley ['w', 'd']
{'w': 1, 'd': 1, 'l': 0}
Tottenham ['w', 'l']
{'w': 1, 'd': 0, 'l': 1}
Everton ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Leicester ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Arsenal ['l', 'w']
{'w': 1, 'd': 0, 'l': 1}
Stoke ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Manchester City ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Crystal Palace ['w', 'l']
{'w': 1, 'd': 0, 'l': 1}
Watford ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Bournemouth ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Manchester United ['w', 'l']
{'w': 1, 'd': 0, 'l': 1}
West Bromwich Albion ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Liverpool ['l', 'd']
{'w': 0, 'd': 1, 'l': 1}
Swansea ['d', 'w']
{'w': 1, 'd': 1, 'l': 0}
{'w': 0, 'd': 0, 'l': 0}
West Ham ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
{'w': 0, 'd': 0, 'l': 0}
Southampton ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Everton ['w', 'w', 'w']
{'w': 3, 'd': 0, 'l': 0}
{'w': 0, 'd': 0, 'l': 0}
Stoke ['w', 'w', 'w']
{'w': 3, 'd': 0, 'l': 0}
Arsenal ['l', 'w', 'd']
{'w': 1, 'd': 1, 'l': 1}
Leicester ['w', 'w', 'w']

In [201]:
# add record for 2018
data_2018 = pd.concat([df_2016, df_2017[columns]], axis = 0)
df_2018['Fixture_id'] = df_2018.index
df_2018['team_history_record'] = df_2018.apply(lambda x: get_win_percentage(x['Fixture_id'], x['oppenet'], df_2018, data_2018), axis = 1)

Huddersfield ['w', 'd']
{'w': 1, 'd': 1, 'l': 0}
Arsenal ['l', 'w', 'd', 'd']
{'w': 1, 'd': 2, 'l': 1}
Newcastle United ['w', 'l']
{'w': 1, 'd': 0, 'l': 1}
Bournemouth ['w', 'w', 'w', 'l']
{'w': 3, 'd': 0, 'l': 1}
{'w': 0, 'd': 0, 'l': 0}
West Ham ['w', 'w', 'l', 'd']
{'w': 2, 'd': 1, 'l': 1}
Liverpool ['l', 'd', 'd', 'w']
{'w': 1, 'd': 2, 'l': 1}
Southampton ['w', 'w', 'w', 'w']
{'w': 4, 'd': 0, 'l': 0}
Manchester United ['w', 'l', 'w', 'l']
{'w': 2, 'd': 0, 'l': 2}
Burnley ['w', 'd', 'l', 'w']
{'w': 2, 'd': 1, 'l': 1}
Crystal Palace ['w', 'l', 'l', 'w']
{'w': 2, 'd': 0, 'l': 2}
Everton ['w', 'w', 'w', 'd']
{'w': 3, 'd': 1, 'l': 0}
Tottenham ['w', 'l', 'w', 'l']
{'w': 2, 'd': 0, 'l': 2}
{'w': 0, 'd': 0, 'l': 0}
{'w': 0, 'd': 0, 'l': 0}
Manchester City ['w', 'w', 'l', 'l']
{'w': 2, 'd': 0, 'l': 2}
Brighton ['w', 'w']
{'w': 2, 'd': 0, 'l': 0}
Leicester ['w', 'w', 'w', 'd']
{'w': 3, 'd': 1, 'l': 0}
Watford ['w', 'w', 'w', 'l']
{'w': 3, 'd': 0, 'l': 1}
Crystal Palace ['w', 'l', 'l', 'w', 

## 4. Model Construction

### 4.1 Feature
The data will look like this

**Team Specific Features**
1. Home {0,1}
2. History record win percentage {0-1}
3. Recent goals number within 5 games{real number}
4. Recent Concede Goals within 5 games{real number}
5. Recent Xg within five games
6. Rank

**Matc Specific Feature**
1. Which part of the season {0,1}

### 4.2 Model outcomes - Bradley-Terry models

The original formula:
$$
Prob(i\;beats\;j) = \frac{P_i}{P_i + P_j}
$$

We set
$$ P_i = e^{\beta_i} \\
logit(Prob(i\;beats\;j)) = \log \frac{Prob(i\;beats\;j)}{Prob(j\;beats\;i)} \\
                         = \log \frac{P_i}{P_j} \\
                         = \beta_i - \beta_j \\$$


#### 4.2.1 Baseline(One feature)

Take example when the only feature is home

$$\beta_i = \lambda \times h_i$$


Here $$\beta_i$$ is the previous beta, while $$\lambda$$ is the parameter we want to estimate and $$h_i$$ is the feature corresponding to whether team_i has home or away game

#### 4.2.2 To multi features

For all the fetures we have, it is easy to expand the **beta** as a linear combiantion of other features **h**

$$
\beta_i = \sum_{k = 1} ^{p} \lambda_k x_{ik}

$$
* Beta is the estmate of performance for team i
* Lambda is the parameter needs to be esimated in the model
* X is the feature value
* P is the number of features we had

For example
$$
logit(Prob(Chelsea\;beats\;Arsenal)) = \beta_{Chelsea} - \beta_{Arsenal} - \beta_{other team} \\
                                    = \sum_{k=1} ^{p} \lambda_{ck} x_{ck} - \sum_{k=1} ^{p} \lambda_{ak} x_{ak}
$$


Note: 
$$
\beta_{other team} = 0 \\
c: Chelsea \\
a: Arsenal
$$




In [220]:
df = pd.concat([df_2017, df_2018], axis = 0)

### 4.3 Adjust the dataframe for the model

In [224]:
df.head()

,id,oppenet,opponent_id,home,result,datetime,xG,num_of_score_goals,num_of_conced_goals,team,Chelsea_rank,opponent_rank,Fixture_id,team_history_record,y
0,7123,Burnley,92,0,l,2017-08-12 15:00:00,1.35651,2,3,Chelsea,7,17,0,0.5,0
1,7137,Tottenham,82,1,w,2017-08-20 16:00:00,0.834423,2,1,Chelsea,15,7,1,0.5,1
2,7140,Everton,72,0,w,2017-08-27 13:30:00,1.16991,2,0,Chelsea,11,7,2,1.0,1
3,7151,Leicester,75,1,w,2017-09-09 15:00:00,1.03943,2,1,Chelsea,6,15,3,1.0,1
4,7167,Arsenal,83,0,d,2017-09-17 13:30:00,0.762129,0,0,Chelsea,3,12,4,0.5,0


In [237]:
df['y'] = df['result'].apply(lambda x: 1 if x == 'w' else 0)
df['recent_goals'] = df['num_of_score_goals'].rolling(5, closed='left').mean()
df['recent_conced'] = df['num_of_conced_goals'].rolling(5, closed='left').mean()
df['recent_state'] = df['y'].rolling(3, closed = 'left').mean()
df['recent_xG'] = df['xG'].rolling(5, closed='left').mean()
df['opponent_history_record'] = 1 - df['team_history_record']

df['datetime'] = pd.to_datetime(df['datetime'])

df.reset_index(inplace=True)


In [244]:
df.loc[:37, 'part'] = df.loc[:37].apply(lambda x: 0 if x['datetime'].year == 2017 else 1, axis = 1)
df.loc[38:, 'part'] = df.loc[38:].apply(lambda x: 0 if x['datetime'].year == 2018 else 1, axis = 1)

In [247]:
df = df[['team', 'home', 'Chelsea_rank', 'team_history_record',
     'recent_goals', 'recent_conced', 'recent_state', 'recent_xG',
     'oppenet', 'opponent_rank', 'opponent_history_record', 'part']]

In [266]:
df = pd.read_csv('/Users/marceloyou/Desktop/Xg-Prediction/data/bsaeline_data/trainign_data_20162017.csv')
df.head(10)

,team,home,Chelsea_rank,team_history_record,recent_goals,recent_conced,recent_state,recent_xG,oppenet,opponent_rank,opponent_history_record,part
0,Chelsea,0,7,0.5,NaN,NaN,NaN,NaN,Burnley,17,0.5,0.0
1,Chelsea,1,15,0.5,NaN,NaN,NaN,NaN,Tottenham,7,0.5,0.0
2,Chelsea,0,11,1.0,NaN,NaN,NaN,NaN,Everton,7,0.0,0.0
3,Chelsea,1,6,1.0,NaN,NaN,0.666667,NaN,Leicester,15,0.0,0.0
4,Chelsea,0,3,0.5,NaN,NaN,1.000000,NaN,Arsenal,12,0.5,0.0
5,Chelsea,1,3,1.0,1.6,1.0,0.666667,1.032480,Stoke,13,0.0,0.0
6,Chelsea,0,3,1.0,2.0,0.4,0.666667,1.080808,Manchester City,1,0.0,0.0
7,Chelsea,1,4,0.5,1.6,0.4,0.333333,1.096230,Crystal Palace,20,0.5,0.0
8,Chelsea,0,5,1.0,1.4,0.8,0.333333,0.992632,Watford,4,0.0,0.0
9,Chelsea,1,4,1.0,1.8,1.0,0.333333,1.141376,Bournemouth,19,0.0,0.0


In [268]:
for col in ['xG','num_of_score_goals', 'num_of_conced_goals']:
    df_2016[col] = df_2016[col].astype('float64')

df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   38 non-null     object 
 1   oppenet              38 non-null     object 
 2   opponent_id          38 non-null     object 
 3   home                 38 non-null     int64  
 4   result               38 non-null     object 
 5   datetime             38 non-null     object 
 6   xG                   38 non-null     float64
 7   num_of_score_goals   38 non-null     float64
 8   num_of_conced_goals  38 non-null     float64
dtypes: float64(3), int64(1), object(5)
memory usage: 2.8+ KB


In [269]:
df["recent_goals"].fillna(df_2016['num_of_score_goals'].mean(),  inplace=True)
df['recent_conced'].fillna(df_2016['num_of_conced_goals'].mean(),  inplace=True)
df['recent_state'].fillna((df_2016['result'].value_counts().loc['w'] / 38), inplace=True)
df['recent_xG'].fillna(df_2016['xG'].mean(),  inplace=True)

In [251]:
df.to_csv('../../../data/bsaeline_data/trainign_data_20162017.csv', index = None)

In [22]:
team_df = pd.read_csv('../../../data/bsaeline_data/trainign_data_20162017.csv')
team_df.head()

,team,home,Chelsea_rank,team_history_record,recent_goals,recent_conced,recent_state,recent_xG,oppenet,opponent_rank,opponent_history_record,part
0,Chelsea,0,7,0.5,NaN,NaN,NaN,NaN,Burnley,17,0.5,0.0
1,Chelsea,1,15,0.5,NaN,NaN,NaN,NaN,Tottenham,7,0.5,0.0
2,Chelsea,0,11,1.0,NaN,NaN,NaN,NaN,Everton,7,0.0,0.0
3,Chelsea,1,6,1.0,NaN,NaN,0.666667,NaN,Leicester,15,0.0,0.0
4,Chelsea,0,3,0.5,NaN,NaN,1.000000,NaN,Arsenal,12,0.5,0.0


### 4.4 Data Spliting and Model part


I will split the data set into training(85%) and test(15%)

In [84]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures

seed = 42

Change the oppnent feature to negative as for the model $$\beta_i - \beta_j$$

In [279]:

df['opponent_rank'] = -df['opponent_rank']
df['opponent_history_record'] = -df['opponent_history_record']

In [6]:
# create dummy variables for column 'oppenent'
team = pd.get_dummies(team_df['oppenet'])

In [284]:
df.drop(columns=['oppenet'], inplace=True)
data = pd.concat([df, team], axis = 1)
data.shape

(76, 33)

In [71]:
df['opponent'] = team_df['oppenet']
df['y'] = team_df['y'].values
df.to_csv('/Users/marceloyou/Desktop/Xg-Prediction/data/bsaeline_data/ready_trainign_data_20162017.csv')

In [74]:
df = pd.read_csv('/Users/marceloyou/Desktop/Xg-Prediction/data/bsaeline_data/ready_trainign_data_20162017.csv', index_col=[0])
df.head()

,team,home,Chelsea_rank,team_history_record,recent_goals,recent_conced,recent_state,recent_xG,opponent_rank,opponent_history_record,...,Southampton,Stoke,Swansea,Tottenham,Watford,West Bromwich Albion,West Ham,Wolverhampton Wanderers,opponent,y
0,Chelsea,0,7,0.5,2.236842,0.868421,0.789474,1.626234,-17,-0.5,...,0,0,0,0,0,0,0,0,Burnley,0
1,Chelsea,1,15,0.5,2.236842,0.868421,0.789474,1.626234,-7,-0.5,...,0,0,0,1,0,0,0,0,Tottenham,1
2,Chelsea,0,11,1.0,2.236842,0.868421,0.789474,1.626234,-7,-0.0,...,0,0,0,0,0,0,0,0,Everton,1
3,Chelsea,1,6,1.0,2.236842,0.868421,0.666667,1.626234,-15,-0.0,...,0,0,0,0,0,0,0,0,Leicester,1
4,Chelsea,0,3,0.5,2.236842,0.868421,1.000000,1.626234,-12,-0.5,...,0,0,0,0,0,0,0,0,Arsenal,0


In [75]:
# create interaction

df['team_part'] = df['part']
oppo_col = team.columns.to_list()
oppo  = ['opponent_rank',
       'opponent_history_record', 'part']

def filter_team(team, li):
       res = [s for s in li if team in s]
       return res


def create_interaction(data, col):
    df = pd.DataFrame(np.zeros((data.shape[0], len(col) * (data.shape[1] - 1))))
    df_col = [i + '_' + j for i in col for j in data.columns.to_list()[:-1]]
    df.columns = df_col
    for i, team in enumerate(data['opponent']):
       col = filter_team(team, df_col)
       print(col)
       for x, y in zip(col, oppo):
              df.loc[i, x] = data.loc[i, y]

    return df
    

AttributeError: 'str' object has no attribute 'columns'

In [77]:
oppo_interaction = create_interaction(df[oppo + ['opponent']], oppo_col)

['Burnley_opponent_rank', 'Burnley_opponent_history_record', 'Burnley_part']
['Tottenham_opponent_rank', 'Tottenham_opponent_history_record', 'Tottenham_part']
['Everton_opponent_rank', 'Everton_opponent_history_record', 'Everton_part']
['Leicester_opponent_rank', 'Leicester_opponent_history_record', 'Leicester_part']
['Arsenal_opponent_rank', 'Arsenal_opponent_history_record', 'Arsenal_part']
['Stoke_opponent_rank', 'Stoke_opponent_history_record', 'Stoke_part']
['Manchester City_opponent_rank', 'Manchester City_opponent_history_record', 'Manchester City_part']
['Crystal Palace_opponent_rank', 'Crystal Palace_opponent_history_record', 'Crystal Palace_part']
['Watford_opponent_rank', 'Watford_opponent_history_record', 'Watford_part']
['Bournemouth_opponent_rank', 'Bournemouth_opponent_history_record', 'Bournemouth_part']
['Manchester United_opponent_rank', 'Manchester United_opponent_history_record', 'Manchester United_part']
['West Bromwich Albion_opponent_rank', 'West Bromwich Albion

In [78]:
data = pd.concat((df[['y','home', 'Chelsea_rank', 'team_history_record', 'recent_goals',
       'recent_conced', 'recent_state', 'recent_xG']], oppo_interaction), axis = 1)
data.shape

(76, 74)

In [80]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]
X.shape, y.shape

((76, 73), (76,))

In [123]:
num = int(X.shape[0] * 0.85)

train_x, train_y = X.iloc[:num], y[:num]
test_x, test_y = X.iloc[num+1:], y[num+1:]

#### 4.5 Model Building

In [129]:
model = LogisticRegression(max_iter=2000)

model.fit(train_x, train_y)

LogisticRegression(max_iter=2000)

In [130]:
pred = model.predict(test_x)

In [131]:
from sklearn.metrics import classification_report

print(classification_report(test_y, pred))

              precision    recall  f1-score   support

           0       0.75      0.50      0.60         6
           1       0.57      0.80      0.67         5

    accuracy                           0.64        11
   macro avg       0.66      0.65      0.63        11
weighted avg       0.67      0.64      0.63        11



In [133]:
!jupyter nbconvert --to html Bradley_Terry_model.ipynb  

[NbConvertApp] Converting notebook Bradley_Terry_model.ipynb to html
[NbConvertApp] Writing 376511 bytes to Bradley_Terry_model.html
